In [1]:
import bz2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gzip
from heapq import heappushpop
from joblib import Parallel, delayed
import time

In [2]:
dataset_ids = []
dataset_vals = []
for data_id in range(10):
    st = time.time()
    dataset_val = []
    with gzip.open('YFCC100M_hybridCNN_gmean_fc6_%d.txt.gz' % data_id , 'rb') as f:
        times = 0
        dist_gt = []
        for line in tqdm(f):
            line = line.decode()
            tokens = line.split('\t')
            if len(tokens) < 2:
                continue
            trn_id = tokens[0]
            trn_val = np.array([float(token) for token in tokens[2:]], dtype=np.float32)
            dataset_ids.append(trn_id)
            dataset_val.append(trn_val)
    dataset_val = np.array(dataset_val)
    dataset_vals.append(dataset_val)
    print(time.time() - st)


1396.6588969230652



1375.059648513794



1384.531615972519



1471.3641579151154



1494.0325946807861



1517.5251760482788



1453.833410024643



1455.7855253219604



1411.2234132289886



1461.141194343567


In [3]:
dataset_ids_set = set(dataset_ids)
file = 'yfcc100m_autotags.bz2'

zero_label = 0
source_file = bz2.BZ2File(file, "r")
times = 0
dataset_label = {}

for line in tqdm(source_file):
    string = line.decode()
    string = string.split('\t')
    key = string[0]
    value = string[1].split(',')
    words = list(map(lambda x: x.split(':')[0], value))
    try:
        probs = list(map(lambda x: x.split(':')[1], value))
    except:
        zero_label += 1
        # not record this into dataset
        continue
    
    if key in dataset_ids_set:
        dataset_label[key] = set(words)
source_file.close()

In [4]:
dataset_y = [dataset_label[x] if x in dataset_label else set() for x in dataset_ids]

In [5]:
x_tst = dataset_vals[0][:100]
y_tst = dataset_y[:100]

x_val = dataset_vals[0][100:1100]
y_val = dataset_y[100:1100]

In [6]:
class X:
    def __init__(self, data, offset):
        self.data = data
        self.offset = offset
    
    def __getitem__(self, key):
        index1 = (key + self.offset) // 1000000
        index2 = (key + self.offset) % 1000000
        return self.data[index1][index2]
    
    def __len__(self):
        l = 0
        for x in self.data:
            l += len(x)
        return l - self.offset
    
x_trn = X(dataset_vals, 1100)
y_trn = dataset_y[1100:]

In [7]:
from tqdm import tqdm_notebook as tqdm
def get_true_KNN(x_trn,x_tst,K):
    N = len(x_trn)
    N_tst = len(x_tst)
    x_tst_knn_gt = np.zeros((N_tst, N))
    for i_tst in tqdm(range(N_tst)):
        dist_gt = np.zeros(N)
        for i_trn in tqdm(range(N)):
            dist_gt[i_trn] = np.linalg.norm(x_trn[i_trn] - x_tst[i_tst], 2)
#             if i_trn % 100000 == 0:
#                 print(i_trn)
        x_tst_knn_gt[i_tst, :] = np.argsort(dist_gt)
    return x_tst_knn_gt.astype(int)

In [8]:
x_val2 = x_val[:100]
y_val2 = y_val[:100]

In [77]:
start = time.time()
x_val_knn_gt = get_true_KNN(x_trn, x_val2, 1)
val_end1 = time.time() - start
print(val_end1)

13425.659036159515


In [78]:
start = time.time()
x_tst_knn_gt = get_true_KNN(x_trn, x_tst, 1)
end1 = time.time() - start
print(end1)

13441.89921927452


In [13]:
val_1nn = x_val_knn_gt[:, 0]
tst_1nn = x_tst_knn_gt[:, 0]

In [19]:
def equal(a, b):
    try:
        return not set.isdisjoint(a, b)
    except KeyError:
        return 0
    
def get_accuracy(y, knn):
    acc = 0
    for i, idx in enumerate(knn):
        acc += int(equal(y[i], y_trn[idx]))
    print(acc / len(knn))
    return acc / len(knn)

In [20]:
get_accuracy(y_val, val_1nn)

0.88


0.88

In [21]:
get_accuracy(y_tst, tst_1nn)

0.9


0.9

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit([label_set])

MultiLabelBinarizer(classes=None, sparse_output=False)

In [11]:
label_set = set()
for i in dataset_y:
    label_set |= i

In [35]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(len(label_set), input_dim=4096))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Using TensorFlow backend.


In [ ]:
tbar = tqdm(range(0, len(x_trn), 32))
for i in tbar:
    x_trn_batch = np.array([x_trn[j] for j in range(i, min(i+32, len(x_trn)))])
    y_trn_batch = mlb.transform(y_trn[i: min(i+32, len(x_trn))])
    msg = model.train_on_batch(x_trn_batch, y_trn_batch)
#     tbar.set_description(model.metrics_names[0] + ': ' + str(msg[0]) + '  \n' + model.metrics_names[1] + ': ' + str(msg[1]))
    tbar.set_description(str(msg[0]))

In [45]:
y_tst_onehot = mlb.transform(y_tst)

In [47]:
y_val_onehot = mlb.transform(y_val)

In [50]:
y_tst_output = model.predict(x_tst)

In [62]:
y_val_output = model.predict(x_val)

In [59]:
def equal(output, ground_truth):
    output[output > 0.5] = 1
    return np.any(output == ground_truth)

In [60]:
def get_accuracy(outputs, truths):
    acc = 0
    for output, truth in zip(outputs, truths):
        acc += int(equal(output, truth))
    return acc / len(outputs)

In [61]:
get_accuracy(y_tst_output, y_tst_onehot)

0.96

In [63]:
get_accuracy(y_val_output, y_val_onehot)

0.964

In [64]:
y_val2_output = model.predict(x_val2)
y_val2_onehot = mlb.transform(y_val2)
get_accuracy(y_val2_output, y_val2_onehot)

0.97

In [65]:
tbar = tqdm(range(0, len(x_trn), 32))
for i in tbar:
    x_trn_batch = np.array([x_trn[j] for j in range(i, min(i+32, len(x_trn)))])
    y_trn_batch = mlb.transform(y_trn[i: min(i+32, len(x_trn))])
    msg = model.train_on_batch(x_trn_batch, y_trn_batch)
#     tbar.set_description(model.metrics_names[0] + ': ' + str(msg[0]) + '  \n' + model.metrics_names[1] + ': ' + str(msg[1]))
    tbar.set_description(str(msg[0]))

In [66]:
y_val_output = model.predict(x_val)
y_val_onehot = mlb.transform(y_val)
get_accuracy(y_val_output, y_val_onehot)

0.966

In [67]:
y_tst_output = model.predict(x_tst)
y_tst_onehot = mlb.transform(y_tst)
get_accuracy(y_tst_output, y_tst_onehot)

0.96

In [68]:
tbar = tqdm(range(0, len(x_trn), 32))
for i in tbar:
    x_trn_batch = np.array([x_trn[j] for j in range(i, min(i+32, len(x_trn)))])
    y_trn_batch = mlb.transform(y_trn[i: min(i+32, len(x_trn))])
    msg = model.train_on_batch(x_trn_batch, y_trn_batch)
#     tbar.set_description(model.metrics_names[0] + ': ' + str(msg[0]) + '  \n' + model.metrics_names[1] + ': ' + str(msg[1]))
    tbar.set_description(str(msg[0]))

In [69]:
y_val_output = model.predict(x_val)
y_val_onehot = mlb.transform(y_val)
get_accuracy(y_val_output, y_val_onehot)

0.97

In [70]:
y_tst_output = model.predict(x_tst)
y_tst_onehot = mlb.transform(y_tst)
get_accuracy(y_tst_output, y_tst_onehot)

0.97

In [89]:
def equal(a, b):
    try:
        return not set.isdisjoint(a, b)
    except KeyError:
        return 0
    
def get_accuracy_multik(y, knn, k=1):
    acc = 0
    for i, idx in enumerate(y):
        correct = False
        for j in range(k):
            correct |= equal(y[i], y_trn[knn[i][j]])
        acc += int(correct)
    print(acc / len(knn))
    return acc / len(knn)

In [90]:
get_accuracy_multik(y_val2, x_val_knn_gt, 1)

0.88


0.88

In [91]:
get_accuracy_multik(y_val2, x_val_knn_gt, 2)

0.94


0.94

In [92]:
get_accuracy_multik(y_val2, x_val_knn_gt, 5)

0.95


0.95

In [93]:
get_accuracy_multik(y_val2, x_val_knn_gt, 3)

0.94


0.94

In [94]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 3)

0.97


0.97

In [95]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 2)

0.96


0.96

In [96]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 1)

0.9


0.9

In [97]:
def equal(a, b):
    try:
        return not set.isdisjoint(a, b)
    except KeyError:
        return 0
    
def compute_single_unweighted_knn_class_shapley(x_trn,y_trn,x_tst_knn_gt,y_tst,K):
    N = len(x_trn)
    N_tst = len(x_tst_knn_gt)
    sp_gt = np.zeros((N_tst,N))
    for j in tqdm(range(N_tst)):
        sp_gt[j,x_tst_knn_gt[j,-1]] = equal(y_trn[x_tst_knn_gt[j,-1]], y_tst[j])/N
        for i in tqdm(np.arange(N-2,-1,-1)):
            sp_gt[j,x_tst_knn_gt[j,i]] = sp_gt[j,x_tst_knn_gt[j,i+1]] +\
                           (equal(y_trn[x_tst_knn_gt[j,i]], y_tst[j]) -
                            equal(y_trn[x_tst_knn_gt[j,i+1]], y_tst[j]))/K* min([K,i+1])/(i+1)
    return sp_gt

In [98]:
K = 2
start2 = time.time()
sp_gt = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, K)
end2 = time.time() - start1

NameError: name 'start1' is not defined

In [99]:
print(end2)

NameError: name 'end2' is not defined

In [109]:
K = 5
start5 = time.time()
sp_gt5 = compute_single_unweighted_knn_class_shapley(x_trn, y_trn, x_tst_knn_gt, y_tst, 5)
end5 = time.time() - start5

In [110]:
print(end5)

6564.723655223846


In [ ]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 5)

In [101]:
def get_accuracy_multik_2(y, knn, k=1):
    acc = 0
    for i, idx in enumerate(y):
        correct = 0
        for j in range(k):
            correct += equal(y[i], y_trn[knn[i][j]])
        acc += int(correct) / k
    print(acc / len(knn))
    return acc / len(knn)

In [102]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 5)

0.98


0.98

In [103]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 2)

0.96


0.96

In [104]:
get_accuracy_multik(y_tst, x_tst_knn_gt, 1)

0.9


0.9

In [105]:
get_accuracy_multik_2(y_tst, x_tst_knn_gt, 1)

0.9


0.9

In [106]:
get_accuracy_multik_2(y_tst, x_tst_knn_gt, 2)

0.915


0.915

In [107]:
get_accuracy_multik_2(y_tst, x_tst_knn_gt, 5)

0.9059999999999997


0.9059999999999997

In [108]:
get_accuracy_multik_2(y_tst, x_tst_knn_gt, 3)

0.9099999999999998


0.9099999999999998

In [113]:
import numpy as np
import pdb


def lsh_function(t,x,w,b):
    # x is 1-d array
    h = np.floor((np.dot(w,x)+b)/t).astype(int)
    return h


class LSH:
    def __init__(self,n_hash_bit,n_hash_table,x_trn,y_trn,t=0.1):
        self.n_hash_bit = n_hash_bit
        self.n_hash_table = n_hash_table
        self.t = t # width of projections
        self.x_trn = x_trn
        self.y_trn = y_trn
        self.N = len(x_trn)
        self.dim = 4096
        # draw w from a normal distribution (2-stable)
        self.w = np.random.normal(0, 1, (n_hash_table, n_hash_bit, self.dim))
        # draw b from U[0,t]
        self.b = np.random.uniform(0, self.t, (n_hash_table, n_hash_bit))
        self.x_trn_hash = [dict() for i in range(n_hash_table)]
        for i in tqdm(range(self.N)):
            hash_code_all = lsh_function(self.t, x_trn[i] / dist_rand, self.w, self.b)
            for l in range(n_hash_table):
                hash_code_trn = '.'.join(map(str, hash_code_all[l, :]))
                if hash_code_trn in self.x_trn_hash[l].keys():
                    self.x_trn_hash[l][hash_code_trn].append(i)
                else:
                    self.x_trn_hash[l][hash_code_trn] = [i]
#             if i % 1000 == 0:
#                 print('build hash %s'%i)

    def get_approx_KNN(self,x_tst,K):
        N_tst = x_tst.shape[0]
        x_tst_knn = np.ones((N_tst, K)) * (-1)
        nns_len = np.zeros(N_tst)
        for i_tst in tqdm(range(N_tst)):
            nns = []
            for l in range(self.n_hash_table):
                hash_code_int = lsh_function(self.t, x_tst[i_tst] / dist_rand, self.w[l, :, :], self.b[l, :])
                hash_code_test = '.'.join(map(str, hash_code_int))
                if hash_code_test in self.x_trn_hash[l].keys():
                    nns += self.x_trn_hash[l][hash_code_test]
            nns = np.unique(nns)
            num_collide_elements = len(nns)
            if len(nns) > 0:
                dist = [np.linalg.norm(self.x_trn[i] / dist_rand - x_tst[i_tst] / dist_rand, 2) for i in nns]
                dist_min_ind = nns[np.argsort(dist)]
                if num_collide_elements < K:
                    x_tst_knn[i_tst, :num_collide_elements] = dist_min_ind[:num_collide_elements]
                else:
                    x_tst_knn[i_tst, :] = dist_min_ind[:K]
            # pdb.set_trace()
            nns_len[i_tst] = len(nns)
            if i_tst % 100 == 0:
                print('get approximate knn %s'%i_tst)
        return x_tst_knn.astype(int),nns_len


    def compute_approx_shapley(self,x_tst_knn,y_tst,K):
        N_tst,K_star = x_tst_knn.shape
        # flag_sufficient = (x_tst_knn[:,-1]>=0)
        sp_approx = np.zeros((N_tst,self.N))
        for j in tqdm(range(N_tst)):
            non_nan_index = np.where(x_tst_knn[j,:]>=0)[0]
            if len(non_nan_index)== 0:
                continue
            K_tot = non_nan_index[-1]
            if K_tot == self.N:
                sp_approx[j, x_tst_knn[j, self.N - 1]] = equal(self.y_trn[x_tst_knn[j, self.N - 1]], y_tst[j]) / self.N
            for i in np.arange(K_tot - 1, -1, -1):
                sp_approx[j, x_tst_knn[j, i]] = sp_approx[j, x_tst_knn[j, i+1]] + (
                        equal(self.y_trn[x_tst_knn[j, i]], y_tst[j]) - equal(
                    self.y_trn[x_tst_knn[j, i + 1]], y_tst[j])) / K * min([K, i + 1]) / (i + 1)



        return sp_approx

In [ ]:
import joblib
lsh = joblib.load('10M/eps0.1/lsh_75_038.pkl') 

In [116]:
np.save('sp_gt5.npy', sp_gt5)

In [117]:
del sp_gt5

In [118]:
import gc
gc.collect()

1037

In [119]:
np.save('10M/eps0.1/sp_gt2.npy', sp_gt)

In [121]:
del sp_gt

In [122]:
np.save('10M/eps0.1/x_tst_knn_gt.npy', x_tst_knn_gt)

In [123]:
del x_tst_knn_gt

In [124]:
np.save('10M/eps0.1/x_val_knn_gt.npy', x_val_knn_gt)

In [125]:
del x_val_knn_gt

In [126]:
import gc
gc.collect()

17